In [1]:
from pyspark.ml.regression import LinearRegression
#from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.datasets import load_diabetes
import pandas
from pyspark.ml.feature import VectorAssembler

In [2]:
diabetes=load_diabetes()
datos = pandas.DataFrame(diabetes.data, columns=diabetes.feature_names)
datos['target'] = pandas.Series(diabetes.target)
datos

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019908,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068330,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005671,-0.045599,-0.034194,-0.032356,-0.002592,0.002864,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022692,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031991,-0.046641,135.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.041708,0.050680,0.019662,0.059744,-0.005697,-0.002566,-0.028674,-0.002592,0.031193,0.007207,178.0
438,-0.005515,0.050680,-0.015906,-0.067642,0.049341,0.079165,-0.028674,0.034309,-0.018118,0.044485,104.0
439,0.041708,0.050680,-0.015906,0.017282,-0.037344,-0.013840,-0.024993,-0.011080,-0.046879,0.015491,132.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044528,-0.025930,220.0


In [3]:
sdatos=spark.createDataFrame(datos)
vectorAssembler = VectorAssembler(inputCols=diabetes.feature_names,outputCol='features')
vdatos = vectorAssembler.transform(sdatos)
datos = vdatos.select(['features', 'target'])
datos.show(n=10,truncate=50,vertical=False)

+--------------------------------------------------+------+
|                                          features|target|
+--------------------------------------------------+------+
|[0.0380759064334241,0.0506801187398187,0.061696...| 151.0|
|[-0.00188201652779104,-0.044641636506989,-0.051...|  75.0|
|[0.0852989062966783,0.0506801187398187,0.044451...| 141.0|
|[-0.0890629393522603,-0.044641636506989,-0.0115...| 206.0|
|[0.00538306037424807,-0.044641636506989,-0.0363...| 135.0|
|[-0.0926954778032799,-0.044641636506989,-0.0406...|  97.0|
|[-0.0454724779400257,0.0506801187398187,-0.0471...| 138.0|
|[0.063503675590561,0.0506801187398187,-0.001894...|  63.0|
|[0.0417084448844436,0.0506801187398187,0.061696...| 110.0|
|[-0.0709002470971626,-0.044641636506989,0.03906...| 310.0|
+--------------------------------------------------+------+
only showing top 10 rows



In [4]:
traindata, testdata = datos.randomSplit([0.7, 0.3])

In [5]:
lr = LinearRegression(
    featuresCol='features', labelCol='target', predictionCol='prediction',
    regParam=0.0, #no regularization
    fitIntercept=True,
    standardization=False,
    loss='squaredError'
)

lr_model = lr.fit(traindata)

print("R squared: {a} \n".format(a=lr_model.summary.r2))

estimate=list(lr_model.coefficients)
estimate.append(lr_model.intercept)

nombres=list(diabetes.feature_names)
nombres.append("intercept")

results=pandas.DataFrame(estimate,columns=["estimate"],index=nombres)
results["p values"]=list(lr_model.summary.pValues)
results

R squared: 0.5355207984451256 



,estimate,p values
age,-34.180772,6.298315e-01
sex,-243.649324,8.153338e-04
bmi,564.185296,4.259282e-11
bp,283.209590,2.507096e-04
s1,-822.881391,8.864054e-02
s2,449.487593,2.476269e-01
s3,128.679213,6.033511e-01
s4,93.358317,6.355943e-01
s5,898.552931,1.271251e-05
s6,56.108523,4.753635e-01
